In [56]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from collections import Counter
import requests
import json

In [57]:
#path for data sets
#vaccination data of us from kaggle
kaggle_vaccine=os.path.join('data','us_state_vaccinations.csv')

#vaccination data from cdc
cdc_vaccine=os.path.join('data','cdc_vaccine.csv')

#demographic and other data from census api
#census_data=os.path.join('data','census_data.csv')

#election data 
#candidate_data=os.path.join('data','president_candidate.csv' )
election_data=os.path.join('data','county_statistics.csv')

In [58]:
#reading csv file from kaggle source
kaggle_vaccine_df=pd.read_csv(kaggle_vaccine)

#reading csv file from kaggle source
cdc_vaccine_df=pd.read_csv(cdc_vaccine)

#reading csv file from census api
#census_data_df=pd.read_csv(census_data)

#reading csv file of election data
#candidate_data_df=pd.read_csv(candidate_data)
election_data_df=pd.read_csv(election_data)

/Users/sudippandey/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
#shape of data
print(f'Shape of data from kaggle source is {kaggle_vaccine_df.shape}')
print(f'Shape of data from cdc source is {cdc_vaccine_df.shape}')
print(f'Shape of data from census api {census_data_df.shape}')
print(f'Shape of election data {election_data_df.shape}')

Shape of data from kaggle source is (18465, 14)
Shape of data from cdc source is (1050251, 32)
Shape of data from census api (33120, 23)
Shape of election data (4867, 51)


In [ ]:
## cleaning datasets

In [43]:
cdc_vaccine_df

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,...,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity
0,10/28/2021,01091,43,Marengo County,AL,49.2,9272,9272.0,57.7,8780,...,D,15.0,16.0,16.0,16.0,Non-metro,7.0,8.0,8.0,8.0
1,10/28/2021,21217,43,Taylor County,KY,48.6,12516,12516.0,57.3,11864,...,C,11.0,12.0,12.0,12.0,Non-metro,7.0,8.0,8.0,8.0
2,10/28/2021,05069,43,Jefferson County,AR,37.7,25225,25199.0,44.0,23518,...,D,14.0,15.0,15.0,15.0,Metro,2.0,3.0,3.0,3.0
3,10/28/2021,30065,43,Musselshell County,MT,31.1,1443,1443.0,35.2,1413,...,B,6.0,6.0,6.0,7.0,Non-metro,6.0,6.0,6.0,7.0
4,10/28/2021,30075,43,Powder River County,MT,19.6,330,330.0,21.9,326,...,A,1.0,1.0,1.0,2.0,Non-metro,5.0,5.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050246,12/13/2020,20205,51,Wilson County,KS,0.0,0,0.0,0.0,0,...,C,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN
1050247,12/13/2020,16031,51,Cassia County,ID,0.0,0,NaN,NaN,0,...,C,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN
1050248,12/13/2020,4013,51,Maricopa County,AZ,0.0,0,0.0,0.0,0,...,C,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN
1050249,12/13/2020,19059,51,Dickinson County,IA,0.0,0,0.0,0.0,0,...,A,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN
